In [237]:
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import summary

In [238]:
def load_trajectory(filename, task):
    """
    This function loads a trajectory from a given file and returns the trajectory and energy data. 
    If the task is 'task_3', it also returns the framework data.

    Parameters:
    filename (str): The name of the file from which to load the trajectory.
    task (str): The task for which the trajectory is being loaded. 
                This should be one of 'task_1', 'task_2', or 'task_3'.

    Returns:
    tuple: Depending on the task, the function returns:
           - (trajectory, energy) for 'task_1' and 'task_2'
           - (trajectory, framework, energy) for 'task_3'
    """
    traj = np.load(filename)
    if task == 'task_1' or task == 'task_2':
        trajectory = traj['trajectory']
        energy = traj['energy']
        return trajectory, energy
    if task == 'task_3':
        trajectory = traj['trajectory']
        framework = traj['framework']
        energy = traj['energy']
        return trajectory, framework, energy

In [239]:
def minimum_image_distance(pos1, pos2, box_length):
    """
    Compute the distance between two points with the minimum image convention.

    Parameters:
    pos1, pos2: numpy arrays representing the positions of the two points.
    box_length: float representing the length of one side of the box.

    Returns:
    float representing the distance between the two points.
    """
    delta = pos2 - pos1
    delta = delta - box_length * np.round(delta / box_length)
    return np.sqrt(np.sum(delta**2))

In [240]:
def create_graph_from_particles(particles, energy, box_length=20.0, cutoff_value=10.0, task='task_1'):
    """
    This function creates a graph representation of a system of particles for use in graph neural networks. 
    Each particle is represented as a node in the graph, and edges are created between particles that are within a certain cutoff distance of each other.

    Parameters:
    particles (numpy.ndarray): A 2D array representing the particles in the system. 
                               Each row represents a particle, with the first two columns representing the x and y coordinates of the particle, 
                               and the remaining columns representing additional features of the particle.
    energy (float): The energy of the system.
    box_length (float, optional): The length of the box in which the particles are contained. 
                                  This is used to calculate the minimum image distance between particles. Defaults to 20.0.
    cutoff_value (float, optional): The maximum distance at which two particles are considered to be connected by an edge. Defaults to 10.0.

    Returns:
    torch_geometric.data.Data: A Data object representing the graph. 
                               The node features, edge indices, edge attributes, and energy are stored as attributes of this object.

    The function first creates a list of node features and a list of edge indices and attributes. 
    Then, it converts these lists into PyTorch tensors and creates a Data object from them. 
    Finally, it validates the Data object to ensure that it is correctly formatted.
    """

    if task == 'task_1':
        num_particles = particles.shape[0]
        x = particles[:, 2:] # Position is not used as node feature

        edge_index = []
        edge_attr = []
        for i in range(num_particles):
            for j in range(num_particles):
                if i != j:  # Avoid self-loops

                    # Append edge if distance is below cutoff
                    pos1 = particles[i, :2]
                    pos2 = particles[j, :2]
                    distance = minimum_image_distance(pos1, pos2, box_length)
                    
                    edge_index.append([i, j])
                    if distance < cutoff_value: # TODO Per essere corretto non bisognerebbe creare l'edge se la distanza è maggiore del cutoff, ma facendo cosi ci sono errori in alcuni casi limite (annuncio canvas). Ho messo quindi rami con pesi negativi (distanze) per gli edge che non dovrebbero esistere. In questo modo il modello dovrebbe imparare a non considerarli.
                        edge_attr.append(distance)
                    else:
                        edge_attr.append(-1.0) # Use -1.0 as padding value

        x = torch.tensor(x, dtype=torch.float)         
        edge_index = torch.tensor(edge_index).t().contiguous()
        edge_attr = torch.tensor(edge_attr, dtype=torch.float)
        energy = torch.tensor(energy, dtype=torch.float)

        # Create Data object
        data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=energy)

        # Validate data object
        data.validate(raise_on_error=True)
        
        return data
    
    elif task == 'task_2':

        num_particles = particles.shape[0]

        x = particles[:, :] # Position is used as node feature
        energies = np.array([energy]*num_particles)
        # Concatenate energy to node features
        # x = np.concatenate((x, energies), axis=1) # TODO check

        edge_index = []
        edge_attr = []
        for i in range(num_particles):
            for j in range(num_particles):
                if i != j:  # Avoid self-loops

                    # Append edge if distance is below cutoff
                    pos1 = particles[i, :2]
                    pos2 = particles[j, :2]
                    distance = minimum_image_distance(pos1, pos2, box_length)
                    
                    edge_index.append([i, j])
                    if distance < cutoff_value: # TODO Per essere corretto non bisognerebbe creare l'edge se la distanza è maggiore del cutoff, ma facendo cosi ci sono errori in alcuni casi limite (annuncio canvas). Ho messo quindi rami con pesi negativi (distanze) per gli edge che non dovrebbero esistere. In questo modo il modello dovrebbe imparare a non considerarli.
                        edge_attr.append(distance)
                    else:
                        edge_attr.append(-1.0) # Use -1.0 as padding value

        x = torch.tensor(x, dtype=torch.float)         
        edge_index = torch.tensor(edge_index).t().contiguous()
        edge_attr = torch.tensor(edge_attr, dtype=torch.float)
        energy = torch.tensor(energy, dtype=torch.float)

        # Create Data object
        # data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
        data = Data(x=x, edge_index=edge_index) # TODO add also edge_attr

        # Validate data object
        data.validate(raise_on_error=True)
        
        return data


## Task 2

In [241]:
def create_data_point(file_path, box_length=20.0, cutoff_value=10.0, task='task_2', max_len=40):
    """
    This function creates a data point from a given trajectory and energy.

    Parameters:
    trajectory (numpy.ndarray): A 2D array representing the trajectory of the system. 
                                The first dimension represents time, and the second dimension represents the x and y coordinates of the particles.
    energy (numpy.ndarray): A 1D array representing the energy of the system at each time step.
    box_length (float, optional): The length of the box in which the particles are contained. Defaults to 20.0.
    cutoff_value (float, optional): The maximum distance at which two particles are considered to be connected by an edge. Defaults to 10.0.

    Returns:
    torch_geometric.data.Data: A Data object representing the graph of the system at each time step.

    The function first creates a list of PyTorch Geometric Data objects, each representing the graph of the system at a different time step. 
    Then, it returns the first element of the list, which corresponds to the first time step.
    """
    trajectory, energy = load_trajectory(file_path, task)
    
    # Create Data object with position, velocity, charge and energy for each time step
    data_point = []
    for i in range(trajectory.shape[0]): # For each time step
        particles = trajectory[i]
        data_point.append(create_graph_from_particles(particles, energy[i], box_length, cutoff_value, task))
            
    return data_point

data_point = create_data_point('data/task1_2/train/trajectory_0.npz')
print(data_point[0])
print(len(data_point))

Data(x=[4, 5], edge_index=[2, 12])
40


In [242]:
class TrajectoryDataset(Dataset):
    """
    A PyTorch Dataset for representing a trajectory of a system of particles.

    Attributes:
    file_paths (list): A list of file paths from which to load the trajectories.
    task (str): The task for which the dataset is being created. This should be one of 'task_1', 'task_2', or 'task_3'.
    max_len (int): The maximum number of particles in the system.
    box_length (float, optional): The length of the box in which the particles are contained. Defaults to 20.0.
    cutoff (float, optional): The maximum distance at which two particles are considered to be connected by an edge. Defaults to 10.0.
    data_list (list): A list of PyTorch Geometric Data objects, each representing the graph of the system at a different time step.
    """
    def __init__(self, file_paths, task='task_2', max_len=40, box_length=20.0, cutoff=10.0):
        """
        Initializes the TrajectoryDataset with the given file paths, task, maximum number of particles, box length, and cutoff distance.
        """
        self.file_paths = file_paths
        self.task = task
        self.max_len = max_len
        self.box_length = box_length
        self.cutoff = cutoff
        self.data_list = []
        for file_path in tqdm(file_paths):
            data_point = create_data_point(file_path, box_length, cutoff, task, max_len)
            self.data_list.append(data_point)
    
    def __len__(self):
        """
        Returns the length of the dataset, i.e., the number of time steps in all trajectories.
        """
        return len(self.data_list)
    
    def __getitem__(self, idx):
        """
        Returns the Data object at the given index.
        """
        trajectory=  self.data_list[idx]
        X = trajectory[0]
        Y = trajectory[1:]
        return X, Y

In [243]:
from sklearn.model_selection import train_test_split
import glob

# Define the paths and parameters
file_paths = glob.glob('data/task1_2/train/*.npz')

max_len = 40  # The maximum length of the trajectories is 40
box_length = 20.0  # The length of the simulation box
cutoff = 10.0  # The cutoff distance for the edges
task = 'task_2'  # The task to perform

train_file_paths, val_file_paths = train_test_split(file_paths, test_size=0.2, random_state=0)

# Create datasets
train_dataset = TrajectoryDataset(train_file_paths, task, max_len, box_length, cutoff)
val_dataset = TrajectoryDataset(val_file_paths, task, max_len, box_length, cutoff)

print(f'Train dataset: {len(train_dataset)} samples')
print(f'Validation dataset: {len(val_dataset)} samples')

100%|██████████| 180/180 [00:00<00:00, 214.51it/s]

Train dataset: 720 samples
Validation dataset: 180 samples


In [244]:
len(train_dataset[0][0])

2

In [245]:
len(train_dataset[0][1])

39

In [246]:
# import torch.nn as nn

# class GRUModel(nn.Module):
#     def __init__(self):
#         super(GRUModel, self).__init__()
#         self.gru1 = nn.GRU(4, 64, batch_first=True)
#         self.repeat_vector = nn.Linear(64, 64)
#         self.gru2 = nn.GRU(64, 64, batch_first=True)
#         self.fc = nn.Linear(64, 4)
    
#     def forward(self, x):
#         # Process the initial step
#         out, _ = self.gru1(x)
#         out = self.repeat_vector(out[:, -1, :]).unsqueeze(1)
#         out = out.repeat(1, 39, 1)
        
#         # Process the repeated vector
#         out, _ = self.gru2(out)
#         out = self.fc(out)
#         return out

class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUModel, self).__init__()
        self.gru1 = nn.GRU(input_size, hidden_size, batch_first=True)
        self.repeat_vector = nn.Linear(hidden_size, hidden_size)
        self.gru2 = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):

        #print(x.shape)
        # Process the initial step
        x = x.unsqueeze(1)  # Add a time step dimension
        #print(x.shape)
        out, _ = self.gru1(x)
        #print(out.shape)
        out = self.repeat_vector(out[:, -1, :]).unsqueeze(1)
        #print(out.shape)
        out = out.repeat(1, 39, 1)
        #print(out.shape)
        
        # Process the repeated vector
        out, _ = self.gru2(out)
        #print(out.shape)
        out = self.fc(out)
        #print(out.shape)
        out = out.squeeze(0)  # Remove the time step dimension
        #print(out.shape)
        return out



In [247]:
class MPNN(nn.Module):
    def __init__(self, num_features, hidden_dim):
        super(MPNN, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
    
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        # Ensure x has the correct shape [num_nodes, num_node_features]
        # Ensure edge_index is of shape [2, num_edges] for undirected graph
        
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        
        # Example global pooling to aggregate node features into a single graph-level representation
        x = torch.mean(x, dim=0, keepdim=True)  # Shape: [1, hidden_dim]
        
        return x

In [248]:
num_features = 5  # Number of features for each node
hidden_dim = 64  # Hidden dimension for the GCN layers
output_size = 5  # Output size for the regression model

In [249]:
embedder = MPNN(num_features=num_features, hidden_dim=hidden_dim)
predictor = GRUModel(hidden_size=hidden_dim, input_size=hidden_dim, output_size=output_size)

graph = train_dataset[0][0]

graph_tensor = embedder(graph)
print(graph_tensor.shape)

torch.Size([1, 64])


In [250]:
# graph_tensor = graph_tensor.unsqueeze(0)
predictions = predictor(graph_tensor)
predictions.shape

torch.Size([39, 5])

In [251]:
class MPNN_GRU(nn.Module):
    def __init__(self, num_features, hidden_dim, output_size):
        super(MPNN_GRU, self).__init__()
        self.embedder = MPNN(num_features, num_features)
        self.predictor = GRUModel(input_size=num_features, hidden_size=hidden_dim, output_size=output_size)
    
    def forward(self, data):
        x = self.embedder(data)
        predictions = self.predictor(x)
        return predictions

In [252]:
model = MPNN_GRU(num_features=num_features, hidden_dim=hidden_dim, output_size=output_size)
predictions = model(graph)
predictions.shape

torch.Size([39, 5])

In [253]:
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for idx in tqdm(range(len(train_dataset))):
        X_batch, Y_batch = train_dataset[idx]
       
        # X_batch = X_batch.unsqueeze(0)  # add a batch dimension TODO

        # Compute embeddings of Y_batch
        Y_batch = torch.stack([model.embedder(graph) for graph in Y_batch])
        
        optimizer.zero_grad()
        output = model(X_batch)
        # loss = criterion(output, Y_batch.unsqueeze(0))  # add a batch dimension for Y_batch
        loss = criterion(output, Y_batch) # TODO
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}')

  0%|          | 0/720 [00:00<?, ?it/s]/Users/lucamainardi/Documents/TUe/DL/DeepLearningAssignment2/.venv/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([39, 1, 5])) that is different to the input size (torch.Size([39, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 720/720 [00:13<00:00, 52.77it/s]


Epoch 1/5, Loss: 0.0158


100%|██████████| 720/720 [00:13<00:00, 54.36it/s]


Epoch 2/5, Loss: 0.0011


100%|██████████| 720/720 [00:13<00:00, 52.96it/s]


Epoch 3/5, Loss: 0.0001


100%|██████████| 720/720 [00:13<00:00, 53.53it/s]


Epoch 4/5, Loss: 0.0000


100%|██████████| 720/720 [00:13<00:00, 55.15it/s]

Epoch 5/5, Loss: 0.0000


In [254]:
def plot_trajectory(trajectory):
    """
    This function plots the trajectory of a system and its energy over time.

    Parameters:
    trajectory (numpy.ndarray): A 2D array representing the trajectory of the system. 
                                The first dimension represents time, and the second dimension represents the x and y coordinates.
    energy (numpy.ndarray): A 1D array representing the energy of the system at each time step.

    The function first plots the trajectory on a 2D grid, with the x and y coordinates on the x and y axes respectively. 
    The initial position is marked in black. The trajectory is represented by a scatter plot, with each point representing the position at a different time step.

    Then, the function plots the energy of the system over time on a separate graph. The x-axis represents the time step, and the y-axis represents the energy.
    """
    
    x = trajectory[...,0]
    y = trajectory[...,1]

    plt.figure(figsize=(4,4))
    plt.vlines([0,20],0,20)
    plt.hlines([0,20],0,20)

    plt.scatter(x[0], y[0], c='black')

    for i in range(x.shape[1]):
        plt.scatter(x[:,i], y[:,i], s=5)

    plt.xlim(-1,21)
    plt.ylim(-1,21)

    plt.show()

In [255]:
test_trajectory = load_trajectory('data/task1_2/train/trajectory_0.npz', 'task_2')[0]
test_trajectory.shape

(40, 4, 5)

In [256]:
test_graph = create_graph_from_particles(test_trajectory[0], 0.0, box_length=20.0, cutoff_value=10.0, task='task_2')
test_graph

Data(x=[4, 5], edge_index=[2, 12])

In [257]:
X = test_graph.x
X.shape

torch.Size([4, 5])

In [258]:
predictions = model(test_graph)
predictions.shape

torch.Size([39, 5])